Importy:

In [ ]:
import torch
from torch import nn
import cv2
import numpy as np
import torchvision
import torchvision.transforms as transforms
import optuna
import matplotlib.pyplot as plt
import numpy as np

Model dyskryminatora:

In [ ]:
class Discriminator(torch.nn.Module):

    def __init__(self):
        super(Discriminator, self).__init__()

        self.conv1 = torch.nn.Conv2d(3, 3, kernel_size = (4, 4), stride = 2, padding="same")
        self.batch_norm = torch.nn.BatchNorm2d(29*3)

        self.activation = torch.nn.ReLU(29*3)
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x